In [1]:
%pip install transformers torch pandas scikit-learn matplotlib
%pip install accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 13.7 MB/s eta 0:00:00:00:0100:01


In [2]:
#Importations et configuration pour Mistral
import pandas as pd
import torch
from transformers import pipeline, set_seed, AutoTokenizer, AutoModelForCausalLM

# --- Configuration du modèle Mistral 7B Instruct ---
MODELE_NOM = "mistralai/Mistral-7B-Instruct-v0.2"

print(f"Chargement du modèle : {MODELE_NOM}...")

# Chargement du Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODELE_NOM)

# Chargement du Modèle avec optimisation 4-bit (pour économiser la VRAM du GPU)
# Necessite les librairies accelerate et bitsandbytes
model = AutoModelForCausalLM.from_pretrained(
	MODELE_NOM,
	load_in_4bit=True,
	dtype=torch.float16, # Utiliser float16 pour optimiser l'utilisation du GPU
	device_map="auto" # Laisse Hugging Face décider où placer les couches (généralement sur le GPU)
)

# Création du Pipeline de génération
generator = pipeline(
	'text-generation', 
	model=model, 
	tokenizer=tokenizer,
	max_new_tokens=50, # Limiter la longueur des réponses pour la concision
	# device=model.device
)

set_seed(42) # Pour rendre les résultats reproductibles
print("Modèle Mistral-7B-Instruct chargé en 4-bit.")

Chargement du modèle : mistralai/Mistral-7B-Instruct-v0.2...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Device set to use cuda:0


Modèle Mistral-7B-Instruct chargé en 4-bit.


In [5]:
#Création du dataset Harry Potter
hp_questions_data = [
	{
		"id": 1,
		"question": "Quel est le nom complet du directeur de Poudlard au début de la série ?",
		"reponse_reference": "Albus Percival Wulfric Brian Dumbledore"
	},
	{
		"id": 2,
		"question": "Quel objet est utilisé pour voyager dans le temps dans le troisième livre ?",
		"reponse_reference": "Retourneur de Temps"
	},
	{
		"id": 3,
		"question": "Combien y a-t-il de Horcruxes créés par Lord Voldemort ?",
		"reponse_reference": "Sept"
	},
	{
		"id": 4,
		"question": "Quel sortilège est le sort de désarmement préféré d'Harry Potter ?",
		"reponse_reference": "Expelliarmus"
	},
	{
		"id": 5,
		"question": "Quel est le nom du chat de Rubeus Hagrid ?", # Question piège (c'est Rusard) mais l'important est la référence
		"reponse_reference": "Miss Teigne"
	},
	{
		"id": 6,
		"question": "Quel est le nom de l'elfe de maison qui servait la famille Malefoy et a été libéré par Harry ?",
		"reponse_reference": "Dobby"
	},
	{
		"id": 7,
		"question": "Dans quelle banque, dirigée par des gobelins, les sorciers gardent-ils leur argent ?",
		"reponse_reference": "Gringotts"
	},
	{
		"id": 8,
		"question": "Qui est le professeur de Potions qui prend la défense contre les forces du mal dans le sixième livre ?",
		"reponse_reference": "Severus Rogue"
	},
	{
		"id": 9,
		"question": "Quelle est la forme du Patronus d'Harry Potter ?",
		"reponse_reference": "Cerf"
	},
	{
		"id": 10,
		"question": "Quelle créature, animal de compagnie d'Hagrid, est condamnée à mort mais sauvée par Harry et Hermione ?",
		"reponse_reference": "Buckbeak"
	}
]

df_hp = pd.DataFrame(hp_questions_data)
print(f"Dataset créé avec {len(df_hp)} questions.")
print(df_hp)

Dataset créé avec 10 questions.
   id                                           question  \
0   1  Quel est le nom complet du directeur de Poudla...   
1   2  Quel objet est utilisé pour voyager dans le te...   
2   3  Combien y a-t-il de Horcruxes créés par Lord V...   
3   4  Quel sortilège est le sort de désarmement préf...   
4   5         Quel est le nom du chat de Rubeus Hagrid ?   
5   6  Quel est le nom de l'elfe de maison qui servai...   
6   7  Dans quelle banque, dirigée par des gobelins, ...   
7   8  Qui est le professeur de Potions qui prend la ...   
8   9   Quelle est la forme du Patronus d'Harry Potter ?   
9  10  Quelle créature, animal de compagnie d'Hagrid,...   

                         reponse_reference  
0  Albus Percival Wulfric Brian Dumbledore  
1                      Retourneur de Temps  
2                                     Sept  
3                             Expelliarmus  
4                              Miss Teigne  
5                                    

In [6]:
#Fonctions pour construire les Prompts

SYSTEM_INSTRUCTION = "Tu es un assistant expert sur l'univers d'Harry Potter. Réponds de manière factuelle et concise."

def formater_prompt_mistral(prompt_utilisateur: str) -> str:
	"""Applique le format d'instruction strict de Mistral."""
	# Le format est : <s>[INST] Instruction [/INST]
	return f"<s>[INST] {prompt_utilisateur} [/INST]"

In [7]:
def creer_prompt_zero_shot(question_actuelle: str) -> str:
	"""Crée un prompt Zero-Shot."""
	
	prompt_utilisateur = (
		f"{SYSTEM_INSTRUCTION}\n\n"
		f"Question: {question_actuelle}\n"
		f"Réponse:"
	)
	
	return formater_prompt_mistral(prompt_utilisateur)

In [8]:
def creer_prompt_one_shot(question_actuelle: str, df_data: pd.DataFrame) -> str:
	"""Crée un prompt One-Shot en utilisant la première entrée du DataFrame comme exemple."""
	
	# Récupérer le premier exemple
	exemple = df_data.iloc[0]
	
	exemples_prompt = (
		f"Question: {exemple['question']}\n"
		f"Réponse: {exemple['reponse_reference']}\n\n"
	)
	
	prompt_utilisateur = (
		f"{SYSTEM_INSTRUCTION}\n\n"
		f"--- Exemple de Format ---\n"
		f"{exemples_prompt}"
		f"--- Fin de l'Exemple ---\n\n"
		f"Question: {question_actuelle}\n"
		f"Réponse:"
	)
	
	return formater_prompt_mistral(prompt_utilisateur)

In [9]:
def creer_prompt_few_shot(question_actuelle: str, df_data: pd.DataFrame, nb_exemples: int = 3) -> str:
	"""Crée un prompt Few-Shot en utilisant les N premières entrées du DataFrame comme exemples."""
	
	# Récupérer les N premiers exemples (s'assurer qu'on ne dépasse pas la taille du DF)
	exemples = df_data.head(nb_exemples)
	
	exemples_prompt = ""
	for index, row in exemples.iterrows():
		# Utiliser le format simple Q:X, R:Y
		exemples_prompt += (
			f"Question: {row['question']}\n"
			f"Réponse: {row['reponse_reference']}\n\n"
		)
		
	prompt_utilisateur = (
		f"{SYSTEM_INSTRUCTION}\n\n"
		f"--- Exemples de Format ({nb_exemples} au total) ---\n"
		f"{exemples_prompt}"
		f"--- Fin des Exemples ---\n\n"
		f"Question: {question_actuelle}\n"
		f"Réponse:"
	)
	
	return formater_prompt_mistral(prompt_utilisateur)